<a href="https://colab.research.google.com/github/sabbasi-1/FineTune-QuestionAnswering-Custom/blob/main/FineTuneTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets evaluate sentencepiece tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


In [ ]:
import transformers,torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,AutoModelForQuestionAnswering,T5ForConditionalGeneration
from transformers import T5Tokenizer, DataCollatorForSeq2Seq

In [ ]:
!pip install -U accelerate

In [ ]:
import json

Extracting data from the json file that had the custom datset.

In [ ]:
datafile='/content/drive/MyDrive/FineTuneCustom/data1.json'
with open(datafile,'r') as file:
  data=json.load(file)

In [ ]:
model_check='google/flan-t5-small'
model=T5ForConditionalGeneration.from_pretrained(model_check)
tokenizer=T5Tokenizer.from_pretrained(model_check)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Converting the data format from list of dictionaries to dictionaries of list so that HuggingFace trainer module can be used as it expects data to be in that format. Furthermore, split the data into training and validation datasets.

In [ ]:
from datasets import Dataset
data_dict = {key: [dic[key] for dic in data] for key in data[0]}
hf_dataset = Dataset.from_dict(data_dict)
split_dataset = hf_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
valid_dataset = split_dataset['test']

In [ ]:
train_dataset['question']

In [ ]:
questions=[]
context=[]
answers=[]
for item in data:
  questions.append(item['question'])
  context.append(item['context'])
  answers.append(item['answer'])

In [ ]:
inputs= ["question: " + question.strip() + " context: " + context for question, context in zip(questions, context)]

In [ ]:
len(inputs)

272

In [ ]:
targets=[answer for answer in answers]

In [ ]:
len(targets)

272

Following are the preprocessing functions for the validation and test datasets. Both functions append questions and contexts together and mark them as the input to the tokenizer. The answers are passed as the targets or labels to the tokenizer. Decoder_input_ids are also included in the whole dataset as flan-t5 model being a encoder-decoder model expects the decoder_input_ids from the both the validation and test data to understand where to start the answer from.

In [ ]:
import torch

def preprocess_for_validation(batch):

    inputs = tokenizer(
        ["question: " + q + " context: " + c for q, c in zip(batch['question'], batch['context'])],
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    model_inputs = {key: inputs[key] for key in ['input_ids', 'attention_mask']}


    if 'answer' in batch:

        targets = tokenizer(
            batch["answer"],
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        model_inputs["labels"] = targets["input_ids"]


        decoder_start_token_id = tokenizer.pad_token_id
        model_inputs["decoder_input_ids"] = torch.cat([
            torch.full((targets["input_ids"].size(0), 1), decoder_start_token_id, dtype=torch.long),
            targets["input_ids"][..., :-1]
        ], dim=1)



    print("Input IDs shape:", model_inputs["input_ids"].shape)
    print("Attention Mask shape:", model_inputs["attention_mask"].shape)
    print("Decoder Input IDs shape:", model_inputs["decoder_input_ids"].shape)
    print("Labels shape:", model_inputs["labels"].shape)


    return model_inputs


In [ ]:
max_length=200
def preprocess_for_flan_t51(batch):

    inputs = tokenizer(
        ["question: " + q + " context: " + c for q, c in zip(batch['question'], batch['context'])],
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )


    targets = tokenizer(
        batch["answer"],
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )


    model_inputs = {key: inputs[key] for key in ['input_ids', 'attention_mask']}
    model_inputs["labels"] = targets["input_ids"]


    decoder_start_token_id = tokenizer.pad_token_id
    decoder_input_ids = [[decoder_start_token_id] + label[:-1].tolist() for label in targets['input_ids']]


    model_inputs["decoder_input_ids"] = torch.tensor(decoder_input_ids, dtype=torch.long)

    print("Input IDs shape:", model_inputs["input_ids"].shape)
    print("Attention Mask shape:", model_inputs["attention_mask"].shape)
    print("Decoder Input IDs shape:", model_inputs["decoder_input_ids"].shape)
    print("Labels shape:", model_inputs["labels"].shape)

    return model_inputs



The datasets being mapped. Both have same features.

In [ ]:
train_data = train_dataset.map(preprocess_for_flan_t51,batched=True)
valid_data= valid_dataset.map(preprocess_for_validation,batched=True)

Map:   0%|          | 0/217 [00:00<?, ? examples/s]

Input IDs shape: torch.Size([217, 200])
Attention Mask shape: torch.Size([217, 200])
Decoder Input IDs shape: torch.Size([217, 200])
Labels shape: torch.Size([217, 200])


Map:   0%|          | 0/55 [00:00<?, ? examples/s]

Input IDs shape: torch.Size([55, 200])
Attention Mask shape: torch.Size([55, 200])
Decoder Input IDs shape: torch.Size([55, 200])
Labels shape: torch.Size([55, 200])


In [ ]:
valid_data

Dataset({
    features: ['question', 'context', 'answer', 'input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
    num_rows: 55
})

In [ ]:
train_data


Dataset({
    features: ['question', 'context', 'answer', 'input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
    num_rows: 217
})

In [ ]:
train_data['decoder_input_ids']

In [ ]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer,Trainer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model_name = model_check.split("/")[-1]


args = Seq2SeqTrainingArguments(
    output_dir=f"Flan-t5-small-custom",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_dir=f"{model_name}-logs",
    logging_steps=25,

    fp16=False,
    push_to_hub=True,

)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=valid_data,
    tokenizer=tokenizer,
)

Trained for a total of 24 epochs in 3 runs

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.410700,2.864750
2,3.188700,2.630311
3,3.020800,2.450727
4,2.896000,2.309308
5,2.789000,2.200236
6,2.674500,2.121452
7,2.612000,2.074732
8,2.555700,2.059332


TrainOutput(global_step=224, training_loss=2.8549234185900008, metrics={'train_runtime': 111.3782, 'train_samples_per_second': 15.587, 'train_steps_per_second': 2.011, 'total_flos': 126056998502400.0, 'train_loss': 2.8549234185900008, 'epoch': 8.0})

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Sabbasi-11/Flan-t5-small-custom")
model = AutoModelForSeq2SeqLM.from_pretrained("Sabbasi-11/Flan-t5-small-custom")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

In [ ]:
question= 'How does Autism Spectrum Disorder affect communication skills?'
context = "Autism Spectrum Disorder (ASD) is a neurodevelopmental disorder that can significantly impact an individual's communication abilities. The effects on communication can vary widely among people with autism. Some individuals might be nonverbal and not use spoken language at all, while others may have extensive vocabularies and speak in long, complex sentences. However, they might still face challenges with social communication. These challenges can include difficulty understanding tone of voice, body language, and facial expressions, as well as interpreting figurative language, such as idioms or humor. Additionally, people with ASD might have a literal interpretation of language and can struggle with back-and-forth conversation, often preferring to talk about their own interests. Early intervention and tailored therapies can greatly assist in developing effective communication skills."
input_text = f"question: {question} context: {context}"



In [ ]:
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=400, truncation=True)

In [ ]:
outputs = model.generate(inputs, max_length=200, num_beams=10, early_stopping=False)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(answer)

Autism Spectrum Disorder (ASD) is a neurodevelopmental disorder that can significantly impact an individual's communication abilities. The effects on communication can vary widely among people with autism. Some individuals might be nonverbal and not use spoken language at all, while others may have extensive vocabularies and speak in long, complex sentences. However, they might still face challenges with social communication. These challenges can include difficulty understanding tone of voice, body language, and facial expressions, as well as interpreting figurative language, such as idioms or humor. Additionally, people with ASD might have a literal interpretation of language and can struggle with back-and-forth conversation, often preferring to talk about their own interests. Early intervention and tailored therapies can greatly assist in developing effective communication skills.
